# Tiền xử lý dữ liệu

Mục tiêu: chúng ta load được các dữ liệu từ các file JSON, sau đó sẽ loại bỏ:
1. Dấu câu, ký tự đặc biệt
2. Stopwords
3. \[Optional\] Chuyển từ viết tắt thành từ đầy đủ


In [1]:
import os,json
import pandas as pd

import unicodedata
import sys

import html

_json&#95;files_: List chứa các file JSON trong data


In [2]:
path_to_news = "Data/"
json_files = [pos_json for pos_json in os.listdir(path_to_news) if pos_json.endswith('.json')]

In [3]:
print(json_files)

['real_1.json', 'fake_1.json']


Tạo dataframe bằng [pandas.DataFrame](http://pandas-docs.github.io/pandas-docs-travis/generated/pandas.Series.loc.html) để có thể lưu các thuộc tính load từ các file JSON đã được load ở trên

In [4]:
news = pd.DataFrame(columns=['title', 'text', 'authors'])

Load dữ liệu từ các file JSON vào list news theo các thuộc tính: _title_, _text_ và _authors_

In [5]:
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_news, js), 'r', encoding='utf-8') as json_file:
        json_text = json.load(json_file)
        
        
        title = json_text['title']
        text = json_text['text']
        author = json_text['authors']
        
        news.loc[index] = [title, text, author]

## 1. Loại bỏ dấu câu

Trong dữ liệu sau khi được crawl về, các ký tự đặc biệt (ví dụ các dấu câu) sẽ được đổi thành dạng [numeric character reference (ncr)](https://en.wikipedia.org/wiki/Numeric_character_reference). Ví dụ như: Trong file _real&#95;1.json_ phần title (ở bên dưới). 

Chúng ta phải đổi lại dưới dạng Unicode để có thể loại bỏ phần dấu câu. 

_html.unescape(str)_: đổi các kí tự dạng __ncr__ về định dạng Unicode của chúng

Tham khảo [stackoverflow](https://stackoverflow.com/a/17018874/5144980)

In [6]:
title_str = news.loc[0]['title']

title_str = html.unescape(title_str)

_remove&#95;punc_: hàm dùng để loại bỏ các kí tự đặc biệt như dấu câu trong định dạng Unicode. Tham khảo: 

In [7]:
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                       if unicodedata.category(chr(i)).startswith('P'))

def remove_punc(text):
    return text.translate(tbl)

__Ví dụ:__ Title của _real&#95;1.json_ 

In [8]:
print(title_str)

Tung tin 'Mr Bean' qua đời để phát tán virus độc hại


Title của _real&#95;1.json_ sau khi loại bỏ kí tự đặc biệt: 

In [9]:
new_string = remove_punc(title_str)
print(new_string)

Tung tin Mr Bean qua đời để phát tán virus độc hại


### Solution

Bắt đầu xử lý các dữ liệu trong list các file JSON và lưu trong list ___preproc&#95;news___:

In [14]:
preproc_news = pd.DataFrame(columns=['title', 'text', 'authors'])

for index, js in enumerate(json_files):
    with open(os.path.join(path_to_news, js), 'r', encoding='utf-8') as json_file:
        json_text = json.load(json_file)
        
        
        title = remove_punc(html.unescape(json_text['title']))
        text = remove_punc(html.unescape(json_text['text']))
        author = json_text['authors']
        
        preproc_news.loc[index] = [title, text, author]
        
print(preproc_news)

                                               title  \
0  Tung tin Mr Bean qua đời để phát tán virus độc...   
1  Thủ tướng Abe cúi đầu xin lỗi vì hành động phi...   

                                                text      authors  
0  Hôm 197 nhiều độc giả cảnh báo nhau trên mạng ...   [Hạ Huyền]  
1  Theo Sankei Sports sáng nay Thủ tướng Nhật Bản...  [Văn Trình]  


## 2. Loại StopWords